In [1]:
from pathlib import Path

SOURCE_GITHUB_REPO_FOLDER = "https://github.com/gabrielecalvo/Language4Water/tree/master/2022-23_semester2"
DESTINATION_FOLDER = Path("original")

In [2]:
# AUXILIARY
import re
from urllib.request import urlopen, urlretrieve
from html.parser import HTMLParser

class MyGithubRepoPageParser(HTMLParser):
    _tag = "a"
    _class = "js-navigation-open Link--primary"
    found = []

    def handle_starttag(self, tag: str, attrs: list[tuple]):
        if tag == self._tag:
            _attribute_dict = dict(attrs)
            if _attribute_dict.get('class') == self._class:
                self.found.append(_attribute_dict['href'])

def _build_dowload_url(github_fld: str):
    match = re.search(r"/(\w+)/(\w+)/blob/(.*)", github_fld)
    user, repo_name, subdir = match.groups()
    download_url = f"https://raw.githubusercontent.com/{user}/{repo_name}/{subdir}"
    return download_url

def _retrive_and_save_file_to_fld(url: str, dst_fld: Path):
    fname = Path(url).name
    urlretrieve(url, dst_fld / fname)

def download_all_repo_fld(src_github_fld: str, dst_local_fld: Path, extensions=(".ipynb",)) -> None:
    with urlopen(src_github_fld) as response:
        body = response.read().decode()
        
    parser = MyGithubRepoPageParser()
    parser.feed(body)
    download_urls = [_build_dowload_url(i) for i in parser.found if i.endswith(extensions)]
    
    dst_local_fld.mkdir(exist_ok=True)
    for url in download_urls:
        _retrive_and_save_file_to_fld(url, dst_fld=dst_local_fld)

In [3]:
download_all_repo_fld(
    src_github_fld=SOURCE_GITHUB_REPO_FOLDER, 
    dst_local_fld=DESTINATION_FOLDER,
)